In [1]:
import sys
argv = sys.argv
if('ipykernel' in argv[0]):
    mod=0
else:
    if(len(argv)==1):
        mod = 0
    else:
        mod = int(argv[1])
print(mod)

database = f"/home/jeon/MissingSat/database"
# database = f"/gem_home/jeon/MissingSat/database"

ncpu=32
memory=None

0


In [2]:
from IPython import get_ipython


def type_of_script():
    """
    Detects and returns the type of python kernel
    :return: string 'jupyter' or 'ipython' or 'terminal'
    """
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'


if type_of_script() == 'jupyter':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize

import numpy as np
import os, glob
os.nice(19)
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
import atexit, signal
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
# import cmasher as cmr
from copy import deepcopy
from multiprocessing import Pool, shared_memory, Value

mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
database = f"/home/jeon/MissingSat/database"

from common_func import *

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [3]:
LG = pklload(f"{database}/LG")
allsubs = None
states = None
for key in LG.keys():
    subs = LG[key]['subs']
    real = LG[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True
    subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'
    state[~ind] = 'pair'
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsubs = subs if allsubs is None else np.hstack((allsubs, subs))
    states = state if states is None else np.hstack((states, state))
argsort = np.argsort(allsubs['id'])
allsubs = allsubs[argsort]
states = states[argsort]

In [4]:
def flush(msg=False, parent=''):
    global memory
    if(msg): print(f"{parent} Clearing memory")
    print(f"\tUnlink `{memory.name}`")
    memory.close()
    memory.unlink()

    signal.signal(signal.SIGINT, signal.SIG_DFL)
    signal.signal(signal.SIGPIPE, signal.SIG_DFL)

def terminate(self, signum):
    flush(msg=True, parent=f'[Signal{signum}]')
    atexit.unregister(flush)
    exit(0)

You know, some properties are missed:  
r200, m200, r200kpc

In [5]:
tree = pklload(f"{database}/02_main_progenitors.pickle")
if(os.path.exists(f"{database}/halo_dict.pickle")):
    halos = pklload(f"{database}/halo_dict.pickle")
else:
    halos = {'catalog':{}, 'index':{}}
    uri.timer.verbose=0
    for iout in tqdm(np.unique(tree['timestep'])):
        isnap = snaps.get_snap(iout)
        ihals = uhmi.HaloMaker.load(isnap, galaxy=False, double_precision=dp)
        indicies = np.zeros(len(ihals), dtype=int)
        iids = tree[tree['timestep'] == iout]['id']
        ihals = ihals[iids-1]
        indicies[iids-1] = np.arange(len(iids))
        halos['catalog'][iout] = ihals
        halos['index'][iout] = indicies   
    pklsave(halos, f"{database}/halo_dict.pickle")
keys = list(halos['index'].keys())
for key in keys:
    if((key%10 != mod)):
        del halos['index'][key]
        del halos['catalog'][key]
keys = list(halos['index'].keys())
nhals = len( halos['catalog'][keys[0]] )

In [6]:
snap_star = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snap_stars = uri.TimeSeries(snap)
snap_dm = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snap_dms = uri.TimeSeries(snap)
snap_cell = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snap_cells = uri.TimeSeries(snap)

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)
[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)
[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [7]:
def calc_virial(cx,cy,cz, rmax_pkpc, pos_code, m_msol, params):
    '''
    input:
        cx,cy,cz : center of halo
        star, dm, cell : data
    output:
        rvir : virial radius
        mvir : virial mass
        rvir_code : virial radius in code unit
    '''
    H0 = params['H0']; aexp=params['aexp']; kpc=params['kpc']
    # critical density
    H02 = (H0 * 3.24078e-20)**2 # s-2
    G = 6.6743e-11 # N m2 kg-2 = kg m s-2 m2 kg-2 = m3 s-2 kg-1
    rhoc = 3 * H02 /8 /np.pi /G # kg m-3
    rhoc *= 5.02785e-31  * (3.086e+19)**3 # Msol ckpc-3
    rhoc /= (aexp**3) # Msol pkpc-3

    # Sorting
    dis = distance3d(pos_code[:,0], pos_code[:,1], pos_code[:,2], cx, cy, cz)/kpc # pkpc
    mask = dis<rmax_pkpc
    argsort = np.argsort(dis[mask])
    dis = dis[mask][argsort] # pkpc
    mas = m_msol[mask][argsort] # Msol

    # Inside density
    cmas = np.cumsum(mas) # Msol
    vols = 4/3*np.pi * dis**3 # pkpc^3
    rhos = cmas / vols # Msol pkpc-3

    arg = np.argmin(np.abs(rhos - 200*rhoc))
    rvir = dis[arg] # pkpc
    # if(rvir>=np.max(dis)):
    #     warnings.warn("rvir is larger than maximum distance!\nEnlarge the box size!")
    # elif(rvir<=np.min(dis)):
    #     warnings.warn("rvir is smaller than minimum distance!\nNot enough particles!")
    # else:
    #     pass
    rvir_code = rvir * kpc # code unit
    mvir = cmas[arg] # Msol
    return rvir, mvir, rvir_code




def calc_virial_mp(hal, kwargs):
    cx,cy,cz = hal['x'], hal['y'], hal['z']
    rmax_pkpc = kwargs['rmax_pkpc']
    pos_code = kwargs['pos_code']
    m_msol = kwargs['m_msol']
    params = {'H0':kwargs['H0'], 'aexp':kwargs['aexp'], 'kpc':kwargs['kpc']}
    rvir, mvir, rvir_code = calc_virial(cx,cy,cz, rmax_pkpc, pos_code, m_msol, params)
    return hal['id'], rvir, mvir, rvir_code

In [8]:
def _ibox(h, factor=1):
    return np.array([[h['x']-factor*h['r'], h['x']+factor*h['r']],
                        [h['y']-factor*h['r'], h['y']+factor*h['r']],
                        [h['z']-factor*h['r'], h['z']+factor*h['r']]])

In [9]:
def _calc_virial(ith, address, shape, dtype):
    global ihals, snap_star, snap_dm, snap_cell, reft, refn, params

    exist = shared_memory.SharedMemory(name=address)
    virials = np.ndarray(shape=shape, dtype=dtype, buffer=exist.buf)

    ihal = ihals[ith]
    r200 = 1000
    factor = 0.5

    while(ihal['r']*factor < r200):
        if(factor>1): print(f'Enlarge the box size! {factor}->{factor*2}')
        factor *= 2
        ibox = _ibox(ihal, factor=factor)
        star = snap_star.get_part_instant(box=ibox, pname='star', target_fields=['x','y','z','m'], nthread=1)
        dm = snap_dm.get_part_instant(box=ibox, pname='dm', target_fields=['x','y','z','m'], nthread=1)
        cell = snap_cell.get_cell_instant(box=ibox, target_fields=['x','y','z','rho', 'level'], nthread=1)

        pos_star = star['pos']; mass_star = star['m','Msol']
        pos_dm = dm['pos']; mass_dm = dm['m','Msol']
        pos_cell = cell['pos']; mass_cell = cell['m','Msol']
        pos_code = np.vstack( (pos_star, pos_dm, pos_cell) )
        mass_msol = np.hstack( (mass_star, mass_dm, mass_cell) )

        
        r200kpc, m200, r200 = calc_virial(ihal['x'], ihal['y'], ihal['z'], factor*ihal['r']/snap_star.unit['kpc'], pos_code, mass_msol, params)

    virials['r200kpc'][ith] = r200kpc
    virials['m200'][ith] = m200
    virials['r200'][ith] = r200

    refn.value += 1
    if(refn.value%100==0)&(refn.value>0):
        print(f" > {refn.value}/{len(virials)} {time.time()-reft.value:.2f} sec (ETA: {(len(virials)-refn.value)*(time.time()-reft.value)/refn.value/60:.2f} min)")

In [10]:
for iout in keys[::-1]:
    ihals = halos['catalog'][iout]
    cpulist = pklload(f"{database}/main_prog/cpulist/cpulist_{iout:05d}.pickle")['all']
    uri.timer.verbose=0
    snap_star = snap_stars.get_snap(iout)
    snap_dm = snap_dms.get_snap(iout)
    snap_cell = snap_cells.get_snap(iout)

    params = {'H0':snap_star.H0,
        'aexp':snap_star.aexp,
        'kpc':snap_star.unit['kpc']}

    virials = np.zeros( nhals, dtype=[("r200kpc","<f8"), ("m200","<f8"), ("r200","<f8")])
    uri.timer.verbose=1
    snap_star.get_part(pname='star', target_fields=['x','y','z','m'], nthread=32, cpulist=cpulist, exact_box=False, domain_slicing=True)
    snap_dm.get_part(pname='dm', target_fields=['x','y','z','m'], nthread=32, cpulist=cpulist, exact_box=False, domain_slicing=True)
    snap_cell.get_cell(target_fields=['x','y','z','rho', 'level'], nthread=32, cpulist=cpulist, exact_box=False, domain_slicing=True)
    uri.timer.verbose=1

    atexit.register(flush)
    signal.signal(signal.SIGINT, terminate)
    signal.signal(signal.SIGPIPE, terminate)
    memory = shared_memory.SharedMemory(create=True, size=virials.nbytes)
    virials = np.ndarray(virials.shape, dtype=virials.dtype, buffer=memory.buf)

    reft = Value('f', 0); reft.value = time.time()
    print(reft.value)
    refn = Value('i', 0)
    uri.timer.verbose=0
    print(f"[IOUT {iout:05d}]")

    with Pool(processes=ncpu) as pool:
        async_result = [pool.apply_async(_calc_virial, (ith, memory.name, virials.shape, virials.dtype)) for ith in range(nhals)]
        iterobj = tqdm(async_result, total=len(async_result), desc=f"IOUT{iout:05d} ")# if(uri.timer.verbose>=1) else async_result
        # iterobj = async_result
        for r in iterobj:
            r.get()
    snap_star.clear()
    snap_dm.clear()
    snap_cell.clear()
    # for i,ihal in tqdm( enumerate(ihals), total=nhals ):
    #     r200 = 1000
    #     factor = 0.5
    #     while(ihal['r']*factor < r200):
    #         # if(factor>1): print(f'Enlarge the box size! {factor}->{factor*2}')
    #         factor *= 2
    #         ibox = _ibox(ihal, factor=factor)
    #         timers = []; ref = time.time()
    #         star = snap_star.get_part_instant(box=ibox, pname='star', target_fields=['x','y','z','m'], nthread=32)
    #         dm = snap_dm.get_part_instant(box=ibox, pname='dm', target_fields=['x','y','z','m'], nthread=32)
    #         cell = snap_cell.get_cell_instant(box=ibox, target_fields=['x','y','z','rho', 'level'], nthread=32)

    #         pos_star = star['pos']; mass_star = star['m','Msol']
    #         pos_dm = dm['pos']; mass_dm = dm['m','Msol']
    #         pos_cell = cell['pos']; mass_cell = cell['m','Msol']
    #         pos_code = np.vstack( (pos_star, pos_dm, pos_cell) )
    #         mass_msol = np.hstack( (mass_star, mass_dm, mass_cell) )

            
    #         r200kpc, m200, r200 = calc_virial(ihal['x'], ihal['y'], ihal['z'], 2*ihal['r']/snap.unit['kpc'], pos_code, mass_msol, params)
    #         # for i in timers:
    #         #     print(f"{i[1]} : {i[0]:.2f}")
    #     virials['r200kpc'][i] = r200kpc
    #     virials['m200'][i] = m200
    #     virials['r200'][i] = r200
    break

Reading 3499 part files (26.73 GiB) in /storage6/NewHorizon/snapshots/output_01020... 
Allocating Memory...
Done (2.999 sec)


Reading parts:   0%|          | 0/3499 [00:00<?, ?it/s]

Done (14.667s).
Reading 3499 part files (26.73 GiB) in /storage6/NewHorizon/snapshots/output_01020... 
Allocating Memory...
Done (4.244 sec)


Reading parts:   0%|          | 0/3499 [00:00<?, ?it/s]

Done (13.066s).
Reading 3499 AMR & hydro files (61.67 GiB) in /storage6/NewHorizon/snapshots/output_01020... 
Allocating Memory...
Done (4.592 sec)


Reading cells:   0%|          | 0/3499 [00:00<?, ?it/s]

Done (27.817s).
1701328896.0
[IOUT 01020]


IOUT01020 :   0%|          | 0/3851 [00:00<?, ?it/s]

 > 100/3851 66.50 sec (ETA: 41.57 min)
 > 200/3851 67.85 sec (ETA: 20.64 min)
 > 300/3851 69.16 sec (ETA: 13.64 min)
 > 400/3851 70.39 sec (ETA: 10.12 min)
 > 500/3851 71.83 sec (ETA: 8.02 min)
 > 600/3851 73.55 sec (ETA: 6.64 min)
 > 700/3851 75.68 sec (ETA: 5.68 min)
[Signal<frame at 0x55ac8f84b070, file '/home/jeon/.conda/envs/jeonpy/lib/python3.10/threading.py', line 320, code wait>] Clearing memory
	Unlink `psm_16bcad02`


In [ ]:
for iout in keys[::-1]:
    ihals = halos['catalog'][iout]
    cpulist = pklload(f"{database}/main_prog/cpulist/cpulist_{iout:05d}.pickle")['all']
    uri.timer.verbose=0
    snap_star = snap_stars.get_snap(iout)
    snap_dm = snap_dms.get_snap(iout)
    snap_cell = snap_cells.get_snap(iout)

    params = {'H0':snap_star.H0,
        'aexp':snap_star.aexp,
        'kpc':snap_star.unit['kpc']}

    virials = np.zeros( nhals, dtype=[("r200kpc","<f8"), ("m200","<f8"), ("r200","<f8")])
    uri.timer.verbose=1
    snap_star.get_part(pname='star', target_fields=['x','y','z','m'], nthread=32, cpulist=cpulist, exact_box=False, domain_slicing=True)
    snap_dm.get_part(pname='dm', target_fields=['x','y','z','m'], nthread=32, cpulist=cpulist, exact_box=False, domain_slicing=True)
    snap_cell.get_cell(target_fields=['x','y','z','rho', 'level'], nthread=32, cpulist=cpulist, exact_box=False, domain_slicing=True)
    uri.timer.verbose=1
    for i,ihal in tqdm( enumerate(ihals), total=nhals ):
        r200 = 1000
        factor = 0.5
        while(ihal['r']*factor < r200):
            # if(factor>1): print(f'Enlarge the box size! {factor}->{factor*2}')
            factor *= 2
            ibox = _ibox(ihal, factor=factor)
            timers = []; ref = time.time()
            star = snap_star.get_part_instant(box=ibox, pname='star', target_fields=['x','y','z','m'], nthread=32)
            dm = snap_dm.get_part_instant(box=ibox, pname='dm', target_fields=['x','y','z','m'], nthread=32)
            cell = snap_cell.get_cell_instant(box=ibox, target_fields=['x','y','z','rho', 'level'], nthread=32)

            pos_star = star['pos']; mass_star = star['m','Msol']
            pos_dm = dm['pos']; mass_dm = dm['m','Msol']
            pos_cell = cell['pos']; mass_cell = cell['m','Msol']
            pos_code = np.vstack( (pos_star, pos_dm, pos_cell) )
            mass_msol = np.hstack( (mass_star, mass_dm, mass_cell) )

            
            r200kpc, m200, r200 = calc_virial(ihal['x'], ihal['y'], ihal['z'], 2*ihal['r']/snap.unit['kpc'], pos_code, mass_msol, params)
            # for i in timers:
            #     print(f"{i[1]} : {i[0]:.2f}")
        virials['r200kpc'][i] = r200kpc
        virials['m200'][i] = m200
        virials['r200'][i] = r200
    break

Reading 3499 part files (26.73 GiB) in /storage6/NewHorizon/snapshots/output_01020... 
Allocating Memory...
Done (3.632 sec)


Reading parts:   0%|          | 0/3499 [00:00<?, ?it/s]

Done (15.139s).
Reading 3499 part files (26.73 GiB) in /storage6/NewHorizon/snapshots/output_01020... 
Allocating Memory...
Done (4.545 sec)


Reading parts:   0%|          | 0/3499 [00:00<?, ?it/s]

Done (14.623s).
Reading 3499 AMR & hydro files (61.67 GiB) in /storage6/NewHorizon/snapshots/output_01020... 
Allocating Memory...
Done (5.696 sec)


Reading cells:   0%|          | 0/3499 [00:00<?, ?it/s]

Done (29.595s).


  0%|          | 0/3851 [00:00<?, ?it/s]

0.8020098209381104 get_star
0.3082289695739746 get_dm
2.2295751571655273 get_cell
0.97493577003479 pos mass
0.15766310691833496 stack
2.4763784408569336 calc
0.014127731323242188 get_star
0.06184196472167969 get_dm
0.5363805294036865 get_cell
0.009055614471435547 pos mass
0.0028319358825683594 stack
0.013330698013305664 calc
0.01997828483581543 get_star
0.03002190589904785 get_dm
0.12219929695129395 get_cell
0.000812530517578125 pos mass
8.916854858398438e-05 stack
0.0008211135864257812 calc
0.023159503936767578 get_star
0.02515387535095215 get_dm
0.059305429458618164 get_cell
0.0013241767883300781 pos mass
5.650520324707031e-05 stack
0.0014033317565917969 calc
0.025086164474487305 get_star
0.09139847755432129 get_dm
0.7032568454742432 get_cell
0.04223275184631348 pos mass
0.0028228759765625 stack
0.07079124450683594 calc
0.009041786193847656 get_star
0.0436244010925293 get_dm
0.301816463470459 get_cell
0.0012259483337402344 pos mass
9.131431579589844e-05 stack
0.0007996559143066406 ca

SystemExit: 0

In [ ]:
stop()
assert not os.path.exists(f"{database}/main_prog/mod{mod}.pickle")
pklsave(["warning"], f"{database}/main_prog/mod{mod}.pickle")

In [ ]:
cursor = 0
nsub = len(allsubs)
uri.timer.verbose=0
for iout in np.unique(tree['timestep'])[::-1]:
    if(os.path.exists( f"{database}/main_prog/virial_{iout:05d}.pickle"))or(iout%10 != mod):
        cursor += nsub
        continue
    isnap = snaps.get_snap(iout)
    ihals = halos['catalog'][iout]
    indicies = halos['index'][iout]

    x1 = np.min(ihals['x']-ihals['r']); x2 = np.max(ihals['x']+ihals['r'])
    y1 = np.min(ihals['y']-ihals['r']); y2 = np.max(ihals['y']+ihals['r'])
    z1 = np.min(ihals['z']-ihals['r']); z2 = np.max(ihals['z']+ihals['r'])
    isnap.box = np.array([[x1,x2],[y1,y2],[z1,z2]])
    globalbox = np.array([[x1,x2],[y1,y2],[z1,z2]])
    uri.timer.verbose=1
    isnap.get_cell(nthread=40, target_fields=['x','y','z','rho','P'], exact_box=False, domain_slicing=False)
    isnap.get_part(nthread=40, target_fields=['x','y','z','m','epoch','id'], exact_box=False, domain_slicing=False)